# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns # Matplotlib 的進階視覺庫
import os
from sklearn.model_selection import train_test_split # 切分資料

data_path = './data'

In [4]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [5]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [7]:
char_to_id = {}
id_to_char = {}

# 建立字典並為訓練數據集中的每個標記分配ID
# 字典應包含 <eos> 和 <pad>
# char_to_id 用於將字符轉換為ID，而 id_to_char 則相反
special_tokens = ['<pad>', '<eos>']

# 包含 src 和 tgt 中的所有字符
all_chars = set(''.join(df_train['src']) + ''.join(df_train['tgt']) + ''.join(df_eval['src']) + ''.join(df_eval['tgt']))

# 分離數字和符號（可選，根據需要）
digits = sorted([char for char in all_chars if char.isdigit()])
symbols = sorted([char for char in all_chars if not char.isdigit()])

# 分配特殊標記的 ID
char_to_id = {'<pad>': 0, '<eos>': 1}

# 分配其他字符的 ID
char_to_id.update({char: idx + 2 for idx, char in enumerate(digits + symbols)})
id_to_char = {idx: char for char, idx in char_to_id.items()}

vocab_size = len(char_to_id)
print('Vocab size: {}'.format(vocab_size))

Vocab size: 18


In [10]:
# 檢查數據表
char_to_id_df = pd.DataFrame(list(char_to_id.items()), columns=['ID', 'Character'])
print(char_to_id_df)

       ID  Character
0   <pad>          0
1   <eos>          1
2       0          2
3       1          3
4       2          4
5       3          5
6       4          6
7       5          7
8       6          8
9       7          9
10      8         10
11      9         11
12      (         12
13      )         13
14      *         14
15      +         15
16      -         16
17      =         17


# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [32]:
df_train = df_train[['src', 'tgt', 'len']]
df_train['char_id_list'] = df_train['src'].apply(lambda x: [char_to_id[char] for char in x] + [char_to_id['<eos>']])
df_train['label_id_list'] = df_train['char_id_list'].apply(lambda x: [char_to_id['<pad>']] * (len(x) - 2) + [0] + x[-2:])
df_train.head()

,src,tgt,len,char_id_list,label_id_list
0,14*(43+20)=882,882,14,"[3, 6, 14, 12, 6, 5, 15, 4, 2, 13, 17, 10, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1]"
1,(6+1)*5=35,35,10,"[12, 8, 15, 3, 13, 14, 7, 17, 5, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 1]"
2,13+32+29=74,74,11,"[3, 5, 15, 5, 4, 15, 4, 11, 17, 9, 6, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1]"
3,31*(3-11)=-248,-248,14,"[5, 3, 14, 12, 5, 16, 3, 3, 13, 17, 16, 4, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 1]"
4,24*49+1=1177,1177,12,"[4, 6, 14, 6, 11, 15, 3, 17, 3, 3, 9, 9, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 1]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [33]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [34]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences[index][0]
        y = self.sequences[index][1]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [35]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']])

In [37]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
dl_eval = torch.utils.data.DataLoader(ds_eval, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [38]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        
        next_char = None
        
        while len(char_list) < max_len: 
            
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively
            
            # 將 char_list 轉換為 tensor
            input_tensor = torch.tensor([char_list], dtype=torch.long)
            
            # 輸入到嵌入層
            embedded = self.embedding(input_tensor)
            
            # 輸入到 LSTM 層
            output, hidden_state1 = self.rnn_layer1(embedded, hidden_state1)
            output, hidden_state2 = self.rnn_layer2(output, hidden_state2)
            
            # 輸入到線性層
            output = self.linear(output)
            
            # 獲取下一個標記的預測值
            y = output[:, -1, :]
            
            # 使用 argmax 函數獲取下一個標記的預測
            next_char = torch.argmax(y, dim=1)
            
            if next_char == char_to_id['<eos>']:
                break
            
            char_list.append(next_char)
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [45]:
torch.manual_seed(2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda':
    print(f"GPU 使用中,", "CUDA 版本: {torch.version.cuda}")
else:
    print("CPU 使用中")

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

GPU 使用中, CUDA 版本: {torch.version.cuda}


In [46]:
import torch.nn.functional as F

# 定義損失函數，使用交叉熵損失，並忽略 <pad> 標記
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

# 定義優化器，使用 Adam 或 AdamW
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 或者使用 AdamW
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [59]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
    
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())
    
    # Evaluate your model
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        
        predictions = # Write your code here. Input the batch_x to the model and generate the predictions
        
        # Write your code here.
        # Check whether the prediction match the ground truths
        # Compute exact match (EM) on the eval dataset
        # EM = correct/total

        
    print(matched/total)

SyntaxError: invalid syntax (2234531792.py, line 34)

# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [54]:
model = model.to("cpu")
print("".join(model.generator('1+1=')))

AttributeError: 'CharRNN' object has no attribute 'generator'